# 3. Models
## Outline

* [Preprocessing](#Preprocessing)
    * [Segmentation](#Segmentation)
    * [tf-idf](#tf-idf)
* [Linear Models](#LinearModels)
* [Binary Logistic Regression Models](#BinaryLogisticRegressionModels)
    * [Cross Validation](#CrossValidation)
* [Exercises and Solutions](#ExercisesAndSolutions)

這一節會教大家建立模型，第一個是用 Logistic Regression 的方法來做分類器，第二個是 Linear Model 來做數值的預測，也會帶大家使用 scikit-learn 這個套件。

## Preprocessing
先將資料做前處理，將新聞的內容斷詞計算詞頻。

In [1]:
import pandas as pd
from pathlib import Path
data_folder = Path("../data/")

news = pd.read_csv(data_folder / "news.csv")
news.head()

,title,content,time,provider,url
0,「把25年前韓國瑜打人事件當英雄看」陳水扁批：吳敦義「災難政治學」的表現,國民黨主席吳敦義日前提到高雄市長候選人韓國瑜過去打陳水扁，表示「很認同跟敬佩」並形容「允文允...,2018-10-22 12:16:02+08:00,風傳媒,https://tw.news.yahoo.com/把25年前韓國瑜打人事件當英雄看-陳水扁...
1,【Yahoo論壇／林青弘】柯文哲是否一再說謊？,柯文哲市長在台北市北投區七星公園造勢，行動競選總部的大卡車開進公園，違規臨停。競辦被開罰六張...,2018-10-22 14:00:26+08:00,林青弘,https://tw.news.yahoo.com/【yahoo論壇%EF%BC%8F林青弘...
2,【Yahoo論壇】民進黨誰最怕陳其邁落選？,讀者投書：廖念漢（現任奇策盟文宣部主任、曾任海巡署專聘講師）\n 《長平之戰》是戰國時代最戲...,2018-10-22 13:57:44+08:00,讀者投書,https://tw.news.yahoo.com/【yahoo論壇】民進黨誰最怕陳其邁落選...
3,抽中籤王 韓國瑜車隊掃街 民眾路邊紛比讚,國民黨高雄市長候選人韓國瑜聲勢上漲，又抽中一號籤王，心情相當興奮，立即展開掃街拜，經過的地方...,2018-10-22 13:32:00+08:00,EBC東森新聞,https://tw.news.yahoo.com/抽中籤王-韓國瑜車隊掃街-民眾路邊紛比讚...
4,百年土地公上香祈福 陳學聖提五不原則,【綜合報導】普悠瑪列車出軌意外舉國震驚如同國難，令社會大眾、競選團隊及陳學聖本人都感到十分沉...,2018-10-22 13:17:44+08:00,民眾日報,https://tw.news.yahoo.com/百年土地公上香祈福-陳學聖提五不原則-0...


In [2]:
news['length'] = news['content'].apply(len)
news['length'].head()

0     627
1    1304
2    1673
3     503
4     585
Name: length, dtype: int64

### Segmentation
使用 jieba 來斷詞

In [3]:
import jieba

In [4]:
text = news.content[0]
print(text)

國民黨主席吳敦義日前提到高雄市長候選人韓國瑜過去打陳水扁，表示「很認同跟敬佩」並形容「允文允武」。前總統陳水扁今（22）日在《新勇哥物語》質疑，這是「災難政治學」的表現，反批吳敦義以為高雄市長贏定了，得意忘形的囂張之情溢於言表。
《新勇哥物語》今天刊出，陳水扁借勇哥表示，不相信吳敦義會講鼓勵暴力的話，也不相信吳敦義主席會認為，當年韓國瑜公然在國會殿堂打阿扁打到住院是對的。
陳水扁指出，吳敦義的發言，是「災難政治學」的表現，他批評吳敦義還真的以為他提名韓國瑜參選高雄市長贏定了，得意忘形的囂張之情溢於言表，竟把25年前的打人事件拿出來捧為文武雙全的英雄看待。「是非不明，黑白不分，不是很可怕嗎？！」
陳水扁也在文中還原，1993年5月韓國瑜推倒他導致受傷住院，隔天有幫派份子聚集到立法院，衝突場面導致10多人受傷掛彩，韓國瑜遭到質疑找黑道兄弟助陣，風波一度越演越烈。韓國瑜後來也道歉，「我願意為我肢體衝突，向陳水扁委員致歉。」
陳水扁表示，當時他是在幫榮民講話，因為他對政府的榮民就養照護問題向退輔會提出質詢，認為「不能把榮民當豬養，不是說榮民是豬」。韓國瑜聽到「豬」就抓狂，這跟扁小時候家裡養豬賣錢供給讀書，生活經驗完全不同。（推薦閱讀：普悠瑪翻車慘劇》吳敦義籲所有九合一選舉候選人暫停選舉活動）
相關報導● 吳敦義因韓國瑜「打扁」才提名選高雄？段宜康酸：乾脆提名殺過人的！● 強力反擊韓國瑜 洪耀福：愛河水臭、自來水不能喝，這是吳敦義當市長時的高雄


In [5]:
print(" ".join(jieba.cut(text)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/80/gbrxkvp9687cyvtdv78_5z3h0000gn/T/jieba.cache
Loading model cost 1.275 seconds.
Prefix dict has been built succesfully.


國民黨 主席 吳敦義 日前 提到 高雄市 長 候選人 韓國瑜 過去 打陳水 扁 ， 表示 「 很 認同 跟 敬佩 」 並 形容 「 允文允武 」 。 前 總統 陳 水扁 今 （ 22 ） 日 在 《 新勇哥 物語 》 質疑 ， 這是 「 災難 政治 學 」 的 表現 ， 反批 吳敦義以 為 高雄市 長 贏定 了 ， 得意忘形 的 囂張 之 情溢 於 言表 。 
 《 新勇哥 物語 》 今天 刊出 ， 陳 水扁 借勇哥 表示 ， 不 相信 吳敦義會 講鼓勵 暴力 的 話 ， 也 不 相信 吳敦義 主席 會 認為 ， 當年 韓國瑜 公然 在 國會 殿堂 打阿扁 打到 住院 是 對 的 。 
 陳 水扁 指出 ， 吳敦義的 發言 ， 是 「 災難 政治 學 」 的 表現 ， 他 批評 吳敦義還 真的 以為 他 提名 韓國瑜 參選 高雄市 長 贏定 了 ， 得意忘形 的 囂張 之 情溢 於 言表 ， 竟 把 25 年前 的 打人 事件 拿出 來 捧 為 文武 雙全 的 英雄 看待 。 「 是非 不明 ， 黑白不分 ， 不是 很 可怕 嗎 ？ ！ 」 
 陳 水扁 也 在 文中 還原 ， 1993 年 5 月 韓國瑜 推倒 他導致 受傷 住院 ， 隔天 有 幫派 份子 聚集 到 立法院 ， 衝突場 面導致 10 多人 受傷 掛彩 ， 韓國瑜 遭到 質疑 找 黑道 兄弟 助陣 ， 風波 一度 越演 越烈 。 韓國瑜後來 也 道歉 ， 「 我願意 為 我 肢體 衝突 ， 向 陳 水 扁委員 致歉 。 」 
 陳 水扁 表示 ， 當時 他 是 在 幫榮民 講話 ， 因為 他 對 政府 的 榮民 就 養照護 問題 向 退 輔會 提出 質詢 ， 認為 「 不能 把 榮民當 豬養 ， 不是 說榮民 是豬 」 。 韓國瑜 聽 到 「 豬 」 就 抓狂 ， 這跟 扁小時 候家裡 養豬 賣 錢 供給 讀書 ， 生活 經驗 完全 不同 。 （ 推薦 閱讀 ： 普悠瑪 翻車 慘劇 》 吳敦義籲 所有 九 合一 選舉候 選人 暫停 選舉 活動 ） 
 相關 報導 ●   吳敦義因 韓國瑜 「 打扁 」 才 提名 選高雄 ？ 段 宜康酸 ： 乾脆 提名 殺過 人 的 ！ ●   強力 反擊 韓國瑜   洪耀福 ： 愛 河水 臭 、 自來 水 不能 喝 ， 這是 吳敦義當 市長 時 的 高雄


In [6]:
news['segmentation'] = news.content.apply(lambda text: " ".join(jieba.cut(text)))
news['segmentation'].head()

0    國民黨 主席 吳敦義 日前 提到 高雄市 長 候選人 韓國瑜 過去 打陳水 扁 ， 表示 「...
1    柯文 哲市 長 在 台北市 北投 區 七星 公園 造勢 ， 行動 競選 總部 的 大卡 車開...
2    讀者 投書 ： 廖念漢 （ 現任 奇策 盟文 宣部 主任 、 曾任 海巡 署 專聘 講師 ）...
3    國民黨 高雄市 長 候選人 韓國瑜 聲勢 上 漲 ， 又 抽中 一號 籤 王 ， 心情 相當...
4    【 綜合 報導 】 普悠瑪 列車 出軌 意外 舉國震 驚 如同 國難 ， 令 社會 大眾 、...
Name: segmentation, dtype: object

### tf-idf
tf: term frequency 詞頻，詞語在單一文本中出現的頻率，
idf: inverse document frequency 逆向檔案頻率，全部文本的數量除以包含詞語的文本的數量  

$\text{tf-idf} = tf * idf$  

例如「的」可能在文本中詞頻高，但是每個文本都有「的」，因此 idf 很小，tf-idf 相乘起來就很小，代表不是重要的訊息

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
news_tfidf = v.fit_transform(news.segmentation)

In [8]:
news_tfidf.shape

(120, 7806)

## Linear Models <a name="LinearModels" />
使用線性的模型來模擬預測未知數值

In [9]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    news_tfidf, 
    news[['length']],
    test_size=0.3, 
    random_state=7)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [11]:
print('Coefficients: \n', regr.coef_)

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [[96.85880969 -2.22508701 14.08190329 ...  0.         36.56082547
   0.        ]]
Mean squared error: 61652.85
Variance score: 0.15


## Binary Logistic Regression Models <a name="BinaryLogisticRegressionModels" />
使用二元分類的模型來預測資料的類別

In [12]:
selected_news = news.loc[news.provider.isin(['中央社','聯合新聞網']), ['content','provider']]
selected_news.head()

,content,provider
14,在競選活動方面，韓流在全台發威，韓國瑜從台灣頭跑到台灣尾進行輔選，他昨天一整天馬不停蹄，為高...,聯合新聞網
15,「台北客家義民嘉年華」重頭戲挑擔踩街活動昨天登場，由5000多人組成的踩街隊伍，一早浩浩蕩蕩...,聯合新聞網
16,市長選戰攻防激烈，台北市長柯文哲卻連連失言，前天酸「台女不化妝上街嚇人」，行動競總「開進」公...,聯合新聞網
32,（中央社記者王揚宇台北21日電）民進黨台北市長參選人姚文智今天在一場論壇說，學校作為有機體，...,中央社
33,（中央社記者黃麗芸台北21日電）「雙北市長青年論壇」今天登場，中國國民黨台北市長參選人丁守中...,中央社


In [13]:
selected_news_tfidf = news_tfidf[selected_news.index]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    selected_news_tfidf, 
    selected_news[['provider']],
    test_size=0.3, 
    random_state=0)

In [15]:
X_train

<18x7806 sparse matrix of type '<class 'numpy.float64'>'
	with 2601 stored elements in Compressed Sparse Row format>

In [16]:
X_test

<9x7806 sparse matrix of type '<class 'numpy.float64'>'
	with 1380 stored elements in Compressed Sparse Row format>

In [17]:
y_train

,provider
96,聯合新聞網
92,聯合新聞網
15,聯合新聞網
72,中央社
119,中央社
109,中央社
58,中央社
49,中央社
33,中央社
94,聯合新聞網


In [18]:
y_test

,provider
16,聯合新聞網
107,中央社
90,聯合新聞網
93,聯合新聞網
34,中央社
73,中央社
99,聯合新聞網
77,中央社
95,聯合新聞網


In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train.provider.values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test.provider.values, lr.predict(X_test))

0.7777777777777778

In [21]:
y_test.provider.values

array(['聯合新聞網', '中央社', '聯合新聞網', '聯合新聞網', '中央社', '中央社', '聯合新聞網', '中央社',
       '聯合新聞網'], dtype=object)

In [22]:
lr.predict(X_test)

array(['中央社', '中央社', '聯合新聞網', '中央社', '中央社', '中央社', '聯合新聞網', '中央社',
       '聯合新聞網'], dtype=object)

### Cross Validation <a name="CrossValidation" />
我們可以使用 Cross Validation 來評估 Classifier 的效果，常用的方法是 k-fold ，也就是將資料分成 k 等份，每次使用其 k-1 份來 training，剩下一份來 testing，總共執行 k 次，這樣做可以充分利用手上已經有的資料來學習。

In [23]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, selected_news_tfidf, selected_news.provider.values, cv=5)
print(scores)

[0.5 1.  1.  0.8 1. ]


In [24]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.86 (+/- 0.39)


## Exercises and Solutions <a name="ExercisesAndSolutions" />

<br>
<details><summary>1. 改用 F1 score 來評定 Classifer 的成效 </summary>
<p>
    
```python
from sklearn.metrics import f1_score
f1_score(y_test.provider.values, lr.predict(X_test), average='macro')
```

</p>
</details>

<details><summary>2. 使用 Multinomial Naive Bayes 來做一個新的 Classifier </summary>
<p>
    
```python
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train.provider.values)
accuracy_score(y_test.provider.values, nb.predict(X_test))
```
</p>
</details>


## More about:  
1. [An introduction to machine learning with scikit-learn](http://scikit-learn.org/stable/tutorial/basic/tutorial.html)
2. [Working With Text Data](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
1. [Scikit Learn User Guide](http://scikit-learn.org/stable/user_guide.html)
